In [60]:
import pandas as pd
import copy
# p.Count, p.Pos, p.CgRank, p.IsLiked, p.IsShared, p.IsProfileOpened, p.IsDownloaded, p.IsFollowed, p.Vplay98
df = pd.read_csv("cg_ranks_big.tsv", sep="\t", header=None, names=["cnt", "pos", "cg_rank", "is_liked", "is_shared", "is_profile_opened", "is_downloaded", "is_followed", "vplay98"])
df

,cnt,pos,cg_rank,is_liked,is_shared,is_profile_opened,is_downloaded,is_followed,vplay98
0,29594,7,creator-affinity-post-backPress,False,False,False,False,False,False
1,39,215,ffm-weighted-vplay98-tf_recall_multilingual_du...,False,False,True,False,False,True
2,3,21,engaged-creator-affinity-post,True,False,True,True,False,True
3,1,55,audio-id-trending-post,False,True,False,True,False,False
4,1,111,ffm-weighted-vplay98-tf_recall_multilingual_du...,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...
240742,1,83,audio-id-affinity-top-post,True,False,True,True,False,False
240743,9,20,lens-affinity-post,True,False,True,False,False,False
240744,18,310,ffm-vplay98-tf_recall_24hr_interaction,False,True,False,False,False,True
240745,116,39,sharecone-weighted-realtimecosine_150k-[1 1 1 ...,False,True,False,True,False,True


In [61]:
# unique cg_ranks
s = set(df["cg_rank"])
len(s), s

(28,
 {'audio-id-affinity-manual-post',
  'audio-id-affinity-top-post',
  'audio-id-trending-post',
  'cluster-affinity-postv2',
  'creator-affinity-post',
  'creator-affinity-post-backPress',
  'engaged-creator-affinity-post',
  'ffm-session-embedding',
  'ffm-vplay98-tf_recall_24hr_interaction',
  'ffm-weighted-play-realtimecosine-[1 1 1 1 1 1]',
  'ffm-weighted-vplay98-tf_recall_multilingual_durationsampling_24hr_60d-[1 1]',
  'interest-creator-affinity-post',
  'latestMinView',
  'lens-affinity-post',
  'promoted',
  'promoted-live-feed',
  'realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0}',
  'realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0}',
  'sharecone-weighted-interaction_2kth_3day-[1]',
  'sharecon

In [55]:
def calc_stats(dt):
    dt = copy.deepcopy(dt)
    dt.groupby(["cg_rank", "pos"]).agg(cnt=pd.NamedAgg(column="cnt", aggfunc=sum)).reset_index()
    dt["cumsum"] = dt.groupby("cg_rank")["cnt"].cumsum()
    dt["cumsum_ratio"] = dt["cumsum"]/(1 + dt["pos"])
    dt = dt.groupby("cg_rank").agg(
        max_pos=pd.NamedAgg(column="pos", aggfunc=max),
        max_cumsum_ratio=pd.NamedAgg(column="cumsum_ratio", aggfunc=max),
        last_cumsum_ratio=pd.NamedAgg(column="cumsum_ratio", aggfunc=lambda ser: ser.values[-1])
    )
    dt.sort_values(by='last_cumsum_ratio', inplace = True, ascending=False)
    return dt

In [62]:
calc_stats(df)

,max_pos,max_cumsum_ratio,last_cumsum_ratio
cg_rank,,,
sharecone-weighted-minview4-[1],9,3.999397e+08,5.013973e+07
sharecone-weighted-lt2k-[1 1 1 1 1],99,6.981411e+08,4.986784e+07
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,1.478759e+09,3.758423e+07
sharecone-weighted-interaction_2kth_3day-[1],499,2.138829e+09,3.122453e+07
promoted-live-feed,0,3.084799e+07,3.084799e+07
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,8.125084e+07,1.015637e+07
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,1.845263e+09,9.926669e+06
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,8.588807e+07,8.783686e+06
sharecone-weightedlag-minview4-[1],9,3.882035e+07,7.765863e+06


In [57]:
dt = df[df["vplay98"]]
calc_stats(dt)

,max_pos,max_cumsum_ratio,last_cumsum_ratio
cg_rank,,,
ffm-session-embedding,99,1.902090e+05,15403.384615
sharecone-weighted-interaction_2kth_3day-[1],499,1.995702e+06,10784.246154
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,2.115700e+04,8195.000000
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,1.834190e+06,5211.741688
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,1.167341e+06,2887.502415
ffm-weighted-vplay98-tf_recall_multilingual_durationsampling_24hr_60d-[1 1],499,2.688800e+04,2783.000000
sharecone-weighted-minview4-[1],9,4.175000e+03,2591.000000
interest-creator-affinity-post,149,7.102300e+04,2480.965517
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,1.516800e+04,847.263158


In [64]:
dt = df[df["is_shared"]]
calc_stats(dt)

,max_pos,max_cumsum_ratio,last_cumsum_ratio
cg_rank,,,
sharecone-weighted-realtimecosine_150k-[1 1 1 1 1 1],499,1.842561e+06,47454.950000
engaged-creator-affinity-post,504,1.503180e+05,30990.200000
sharecone-weighted-interaction_2kth_3day-[1],499,2.781209e+06,18275.516340
realTimeLabelD_overall_top {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,1.390210e+05,17393.125000
sharecone-weighted-multilingual_2kth_3d_plc-[1 1],499,2.223015e+06,12122.497326
realTimeLabelD_overall_hof {Like:1 Share:3 VideoWatch:0 TimeSpent:1 Percentile70:0 ProfileOpen:3 CommentCreated:0 FollowUser:6 Repeat_95:0 Repeat1_5:0 NegativeSignal:0 Download:3 Dislike:0},29,1.172760e+05,11896.200000
sharecone-weighted-lt2k-[1 1 1 1 1],99,2.731550e+05,9755.714286
sharecone-weighted-minview4-[1],9,6.589800e+04,8240.750000
ffm-session-embedding,99,4.774110e+05,5694.188889
